In [16]:
import collections
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import re
import requests
import time
import xml.etree.ElementTree

st_accept = "text/html"
st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
headers = {
   "Accept": st_accept,
   "User-Agent": st_useragent
}

NIIKM_BASE_LINK = "https://www.niikm.ru/"
NIIKM_LINKS = ["products/azot/"]
PANDAS_CRUTCH_BASE = "/content/temp"
REQUEST_DELAY = 3

In [ ]:
def parse_url(url):
    print("Parsing html from \"" + url + "\".")

    req = requests.get(url, headers)

    if req.status_code != 200:
        print("    ERROR: The request status is " + str(req.status_code) + ".")
        return

    soup = bs(req.text, 'lxml')
    #processed_tags = set()

    #A crutch for Pandas library.
    PANDAS_CRUTCH = PANDAS_CRUTCH_BASE + str(id(url)) + ".txt"
    temp_file = open(PANDAS_CRUTCH, 'w', encoding="utf-8")
    temp_file.write(req.text)
    temp_file.close()
    tables = pd.read_html(PANDAS_CRUTCH)
    table_index = 0

    page_title = soup.find("div", attrs={"class":"title-page"})
    if page_title == None:
        print("    ERROR: Could not find page title.")
        return

    f = open(page_title.text + ".txt", 'w', encoding="utf-8")
    f.write(page_title.text.lstrip().rstrip().replace("\n","") + "\n")

    #Required for distinguishing articles and product pages.
    is_product_page = True
    product_description = soup.find("div", attrs={"class":"product-card-dbl__col product-card-dbl__description-product"})
    if product_description == None:
        is_product_page = False

    if is_product_page:
        print("    INFO: The page is a product page.\n    Parsing product description...")
        for i in product_description.children:
            if i.name == "p":
                f.write(i.text.lstrip().rstrip() + "\n")
        print("        Done.")

        product_assortment = soup.find("div", attrs={"class":"product-card-dbl__product-range-wrap"})
        if product_assortment == None:
            print("    INFO: The page does not contain references to the product subtypes.")
        else:
            print("    Starting the recursive traversal of the product subtypes...")
            for i in product_assortment:
                time.sleep(REQUEST_DELAY)

                if i.name == "a":
                    parse_url(NIIKM_BASE_LINK + i["href"])

            print("        Done.")
    else:
        #TODO: add code for articles.
        pass

    f.close()
    os.remove(PANDAS_CRUTCH)
    print("    Done.")

for i in NIIKM_LINKS:
    parse_url(NIIKM_BASE_LINK + i)
    time.sleep(REQUEST_DELAY)
